In [1]:
import tensorflow as tf
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
import keras

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import BatchNormalization
from keras import backend as K
from keras.layers import Dropout
K.set_image_data_format('channels_first')
from keras.layers import concatenate
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import LSTM
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Conv2D
from keras.optimizers import SGD,RMSprop,Adam
import glob
from tensorflow.keras.applications import VGG19

In [2]:
#Assigning Data Path and Labels
PATH = os.getcwd()
# Define data path
data_path = 'F:\Advanced Machine Learning\Test dataset'
 
data_dir_list = os.listdir(data_path)

img_rows=128
img_cols=128
num_channel=3
no_images=0

for dataset in data_dir_list:
    img_list = os.listdir(data_path + '/' + dataset)
    no_images = no_images+len(img_list)

# Define the number of classes
labels = np.ones((no_images,),dtype='int64')
num_classes = 8
label_index=0
img_data_list=[]
img=0

for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img)
        input_img_resize=cv2.resize(input_img,(128,128))
        img_data_list.append(input_img_resize)
        if dataset[0]== 'AK':
            labels[label_index]= 0
            #print(dataset[0])
        if dataset[0] == 'BCC':
            labels[label_index] = 1
            #print(dataset[0])
        if dataset[0] == 'BKL':
            labels[label_index] = 2
        label_index = label_index+1
        #print(dataset[0])
        if dataset[0] == 'DF':
            labels[label_index] = 3
        label_index = label_index+1
        #print(dataset[0])
        if dataset[0] == 'MEL':
            labels[label_index] = 4
        label_index = label_index+1
        #print(dataset[0])
        if dataset[0] == 'NV':
            labels[label_index] = 5
        label_index = label_index+1
        #print(dataset[0])
        if dataset[0] == 'SCC':
            labels[label_index] = 6
        label_index = label_index+1
        #print(dataset[0])
        if dataset[0] == 'VASC':
            labels[label_index] = 7
        label_index = label_index+1
        #print(dataset[0])

Loaded the images of dataset-AK

Loaded the images of dataset-BCC

Loaded the images of dataset-BKL

Loaded the images of dataset-DF

Loaded the images of dataset-MEL

Loaded the images of dataset-NV

Loaded the images of dataset-SCC

Loaded the images of dataset-VASC



In [3]:
#Image Preprocessing
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255
print (img_data.shape)

(1402, 128, 128, 3)


In [4]:
if num_channel==1:
    if K.set_image_data_format('channels_last'):
        img_data= np.expand_dims(img_data, axis=1)
        print (img_data.shape)
    else:
        img_data= np.expand_dims(img_data, axis=3)
        print (img_data.shape)

else:
    if K.set_image_data_format('channels_last'):
        img_data=np.rollaxis(img_data,3,1)
        print (img_data.shape)


        labels[0:198] = 0
        labels[199:402] = 1
        labels[403:609] = 2
        labels[610:705] = 3
        labels[706:882] = 4
        labels[883:1098] = 5
        labels[1099:1274] = 6
        labels[1275:1402] = 7     
        print(X_train.shape)

In [5]:
# Assigning Labels

# Define the number of classes
num_classes = 8

names = ['AK','BCC','BKL', 'DF', 'MEL', 'NV', 'SCC', 'VASC']

# convert class labels to one-hot encoding
Y = np_utils.to_categorical(labels, num_classes)

#Shuffle the dataset with random state=2
X,Y = shuffle(img_data, Y, random_state=2)
# Split the dataset with 20% testing data
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,
    test_size=0.2,
)

print("X_train size is {} and y_train size is {}".format(X_train.shape,Y_train.shape))
print("X_test size is {} and y_test size is {}".format(X_test.shape,Y_test.shape))

X_train size is (1121, 128, 128, 3) and y_train size is (1121, 8)
X_test size is (281, 128, 128, 3) and y_test size is (281, 8)


In [6]:
from tensorflow.keras.applications import *

In [7]:
model = ResNet152(include_top=False)

In [8]:
model.summary()

Model: "resnet152"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
__________________________________________________________________________________________

In [9]:
conv_start = tf.keras.applications.resnet.ResNet152(
    include_top=False, weights='imagenet'
)

In [10]:
from keras.layers import Dense, Flatten
from keras.models import Sequential

model = Sequential()
model.add(conv_start)
model.add(Flatten(input_shape=(128, 128, 3)))
model.add(Dense(25, activation='relu'))
model.add(Dropout(0.30))
model.add(Dense(8, activation='softmax'))


In [11]:
for layer in conv_start.layers:
    layer.trainable = False

In [12]:
learning_rate = 0.001
opt = RMSprop(lr=learning_rate)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

C:\Users\dawoo\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [13]:
# train the model
def evaluate_this_model(model, epochs):
    
    np.random.seed(1)

    history = model.fit(X_train, Y_train,epochs=epochs)
    results = model.evaluate(X_test, Y_test)
    
    
    print("\n\nAccuracy on training set is {}".format(history.history["accuracy"][-1]))
    print("\nAccuracy on test set is {}".format(results[1]))
# # score = None[0]
# print('Test Loss:', score[0])
# print('Test accuracy:', score[1])
evaluate_this_model(model, 5)

Epoch 1/5
36/36 [==============================] - 141s 3s/step - loss: 0.3666 - accuracy: 0.8723
Epoch 2/5
36/36 [==============================] - 114s 3s/step - loss: 0.0225 - accuracy: 0.9935
Epoch 3/5
36/36 [==============================] - 110s 3s/step - loss: 0.0109 - accuracy: 0.9934
Epoch 4/5
36/36 [==============================] - 108s 3s/step - loss: 0.0142 - accuracy: 0.9912
Epoch 5/5
9/9 [==============================] - 31s 3s/step - loss: 0.0000e+00 - accuracy: 1.0000


Accuracy on training set is 0.9982158541679382

Accuracy on test set is 1.0
